# 电商用户行为分析案例--天池数据集User Behavior Data from Taobao

# 1 项目背景与数据集

## 用户行为分析过程
说明
本次数据分析基于阿里云天池数据集(用户行为数据集)，使用转化漏斗，对常见电商分析指标，包括转化率，PV，UV，复购率等进行分析，分析过程中使用Python进行数据的清洗，清洗后进行数据分析及可视化。


## 数据集及样本选择

该数据集记录用户在淘宝网站浏览商品产生的行为数据。数据集包含了2017年11月25日至2017年12月3日之间，约一百万随机用户的所有行为（行为包括点击、购买、加购、喜欢）。数据集大小情况为：用户数量约100万（987,994），商品数量约410万（4,162,024），商品类目数量9,439以及总的淘宝用户行为记录数量为1亿条（100,150,807）。我们为了便于理解问题和提高数据处理速度，只选取数据集中的3000万条记录进行数据分析。

## 提出问题
1、整体用户的购物情况
PV（总访问量）、日均访问量、UV（用户总数）、有购买行为的用户数量、用户的购物情况、复购率分别是多少？

2、用户行为转化漏斗
点击-加购物车-收藏-购买各环节转化率如何？购物车遗弃率是多少，如何提高？

3、购买率高和购买率为0的人群有什么特征？

4、基于RFM模型的用户分析

5、商品销售的一些情况


# 2 清洗后数据读入Pandas

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'.\data\UserBehavior_Done.csv', encoding='gbk')
df.tail()


,User_Id,Item_Id,Category_Id,Behavior_type,Date,Time
2998527,218275,2772813,1577687,pv,2017-11-29,23:10:10
2998528,218275,4654492,1577687,pv,2017-11-29,23:11:08
2998529,218275,2772813,1577687,pv,2017-11-29,23:11:17
2998530,218275,1207527,1577687,pv,2017-11-29,23:12:50
2998531,218275,1525621,1577687,pv,2017-11-29,23:13:20


# 3 构建模型和分析问题
## 3.1 用户流量及购物情况

### (1) pv（总访问量）

In [2]:
#总访问量
# pv (总访问量):2684410  # fav(喜欢)：86576   # cart(加购物车)：167572  # buy(购买)：59974
df[df['Behavior_type'] == 'pv']['Behavior_type'].count()  

np.int64(2684410)

 - `df['Behavior_type']`：这指的是DataFrame中名为`'Behavior_type'`的一列，该列存储了不同用户行为类型的标识符，例如'pv', 'fav', 'cart', 'buy'等。

- `df[df['Behavior_type'] == 'pv']`：这部分代码应用了一个布尔索引，筛选出所有`'Behavior_type'`列为'pv'的行，即所有表示页面浏览行为的记录。

- `df[df['Behavior_type'] == 'pv']['Behavior_type'].count()`：最后，对筛选出来的表示页面浏览行为的所有记录，计算其数量。

### (2) 日均访问量

In [3]:
# 日均访问量
df[df['Behavior_type'] == 'pv'].groupby('Date')[['Behavior_type']].count()

,Behavior_type
Date,
2017-11-25,278615
2017-11-26,285850
2017-11-27,267000
2017-11-28,265157
2017-11-29,271342
2017-11-30,280678
2017-12-01,294298
2017-12-02,370282
2017-12-03,371188


### (3) uv（用户总数）

In [4]:
# -- 用户总数:29178
len(df['User_Id'].unique())

29178

### (4) 有购买行为的用户数量

In [5]:
# 有购买行为的用户数量:19902
len(df[df['Behavior_type'] == 'buy']['User_Id'].unique())  #有购买行为的用户数量19902
#等价于
df[df['Behavior_type'] == 'buy']['User_Id'].nunique()  #nunique()返回的是唯一值的个数

19902

**`.nunique()`**：这是 Pandas 提供的一个方法，直接计算 Series（这里是 `'User_Id'` 列）中的唯一值数量。

### (5) 用户的购物情况

In [6]:
# -- 用户的购物情况
df_pivot = df.pivot_table(values='Item_Id',index='User_Id',columns='Behavior_type',aggfunc='count',fill_value=0)
df_pivot.head()

Behavior_type,buy,cart,fav,pv
User_Id,,,,
1,0,0,0,55
13,0,4,0,7
19,0,1,0,74
21,0,1,6,329
100,8,0,6,84


这段代码是用来创建一个透视表（Pivot Table），以展示不同用户在不同行为类型下的活动情况。

1. **`df.pivot_table(...)`**:
   - 这是 Pandas 中用于创建透视表的方法。透视表可以将数据按照不同的维度进行聚合和展示。

2. **`values='Item_Id'`**:
   - 指定了透视表中的值字段为 `'Item_Id'`。这意味着透视表中的每个单元格将显示与 `'Item_Id'` 相关的统计数据。

3. **`index='User_Id'`**:
   - 指定了透视表的行索引为 `'User_Id'`。这意味着每一行将代表一个不同的用户。

4. **`columns='Behavior_type'`**:
   - 指定了透视表的列索引为 `'Behavior_type'`。这意味着每一列将代表一种不同的行为类型。

5. **`aggfunc='count'`**:
   - 指定了对每个单元格的聚合方式为计数（`'count'`）。这意味着每个单元格将显示每个用户在每种行为类型下的活动次数。

6. **`fill_value=0`**:
   - 当某个用户没有执行某种行为类型时，默认情况下透视表中的相应单元格将是缺失值（NaN）。`fill_value=0` 参数设置这些缺失值为 0。

### (6) 复购率：产生两次或两次以上购买的用户占购买用户的比例

In [7]:
df_buy1 = pd.DataFrame({'购买数大于1次':[sum(df_pivot['buy']>1)]})
df_buy1

,购买数大于1次
0,13124


In [8]:
购买数大于1次 = sum(df_pivot['buy']>1)
购买数大于1次

13124

In [9]:
总购买数 = sum(df_pivot['buy']>0)
总购买数 

19902

In [10]:
复购率 = "{:.2%}".format(购买数大于1次/总购买数)
复购率

'65.94%'

## 3.2 用户行为转化漏斗
在购物环节中收藏和加入购物车两个环节没有先后之分，所以将这两个环节放在一起作为购物环节的一步。最终得到用户购物行为各环节转化率，如下：

In [13]:
#统计每种行为类型 下的用户数量
df_behavior_numbers=df.pivot_table(values='User_Id', index='Behavior_type', aggfunc='count')
df_behavior_numbers

,User_Id
Behavior_type,
buy,59974
cart,167572
fav,86576
pv,2684410


In [ ]:
# 这个转化率需要跟行业或者竞争对手的数据进行对比分析，也可以根据自己店铺的历史数据，
# 或者不同商品进行对比，当出现异常时，回溯流程进而发现问题。
pv=df_behavior_numbers.loc['pv']/df_behavior_numbers.loc['pv']
pv_to_buy=df_behavior_numbers.loc['buy']/df_behavior_numbers.loc['pv']
pv_to_favCart=(df_behavior_numbers.loc['cart']+df_behavior_numbers.loc['fav'])/df_behavior_numbers.loc['pv']
# pv             #点击
# pv_to_buy      #点击转购买
# pv_to_favCart  #点击转收藏
print(pv)
print(pv_to_buy)
print(pv_to_favCart)

User_Id    1.0
Name: pv, dtype: float64
User_Id    0.022342
dtype: float64
User_Id    0.094676
dtype: float64



- `df_behave_numbers.loc['pv']/df_behave_numbers.loc['pv']`:
    - 将 `'pv'` 行为类型的用户数量除以自己，结果为 1.0。这是因为任何非零数字除以自身都等于 1。

- `df_behave_numbers.loc['buy']/df_behave_numbers.loc['pv']`:
    - 计算购买行为类型的用户数量与点击行为类型的用户数量之比。

- `(df_behave_numbers.loc['cart']+df_behave_numbers.loc['fav'])/df_behave_numbers.loc['pv']`:
    - 计算加购行为类型和收藏行为类型的用户数量之和与点击行为类型的用户数量之比。

In [16]:
import pyecharts.options as opts
from pyecharts.charts import Funnel

x_data = ["点击", "收藏+购物车", "购买"]
y_data = [pv[0], pv_to_favCart[0], pv_to_buy[0]]

data = [[x_data[i], y_data[i]] for i in range(len(x_data))]

(
    Funnel(init_opts=opts.InitOpts())
    .add(
        series_name="",
        data_pair=data,
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} : {c}%"),
        label_opts=opts.LabelOpts(is_show=True, position="inside"),
        itemstyle_opts=opts.ItemStyleOpts(border_color="#fff", border_width=1),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="漏斗图"))
    .render_notebook()
)



C:\Users\33243\AppData\Local\Temp\ipykernel_13724\3025578921.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_data = [pv[0], pv_to_favCart[0], pv_to_buy[0]]


这段代码使用 PyEcharts 库来创建一个漏斗图，用于可视化不同行为类型之间的转换率。

1. **导入必要的模块**:
   ```python
   import pyecharts.options as opts
   from pyecharts.charts import Funnel
   ```
   - 导入 `pyecharts.options` 作为 `opts`，这是用于配置图表选项的模块。
   - 从 `pyecharts.charts` 导入 `Funnel` 类，用于创建漏斗图。

2. **定义 X 轴数据**:
   ```python
   x_data = ["点击", "收藏+购物车", "购买"]
   ```
   - 定义了一个列表 `x_data`，包含了漏斗图 X 轴上的标签，即不同阶段的名称。

3. **定义 Y 轴数据**:
   ```python
   y_data = [pv[0], pv_to_favCart[0], pv_to_buy[0]]
   ```
   - 定义了一个列表 `y_data`，包含了与 `x_data` 中各个阶段相对应的数值。这些数值分别是从之前的代码片段中得到的转换率：
     - `pv[0]`: 1.0，点击行为类型的用户数量与自身的比率（通常作为基准）。
     - `pv_to_favCart[0]`: 从点击到收藏和加购行为类型的转换率。
     - `pv_to_buy[0]`: 从点击到购买行为类型的转换率。

4. **组合 X 和 Y 轴数据**:
   ```python
   data = [[x_data[i], y_data[i]] for i in range(len(x_data))]
   ```
   - 使用列表推导式将 `x_data` 和 `y_data` 组合成一个二维列表，其中每个子列表包含两个元素：X 轴标签和相应的 Y 轴值。

5. **创建漏斗图**:
   ```python
   (
       Funnel(init_opts=opts.InitOpts())
       .add(
           series_name="",
           data_pair=data,
           tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} : {c}%"),
           label_opts=opts.LabelOpts(is_show=True, position="inside"),
           itemstyle_opts=opts.ItemStyleOpts(border_color="#fff", border_width=1),
       )
       .set_global_opts(title_opts=opts.TitleOpts(title="漏斗图"))
       .render_notebook()
   )
   ```
   - 使用 `Funnel` 类创建一个漏斗图实例，并传入初始化选项 `init_opts`。
   - 使用 `.add()` 方法添加数据系列，指定：
     - `series_name=""`: 不显示系列名称。
     - `data_pair=data`: 传入之前组合好的数据。
     - `tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} : {c}%")`: 配置工具提示选项，当鼠标悬停在漏斗图上时显示的信息格式。
     - `label_opts=opts.LabelOpts(is_show=True, position="inside")`: 显示标签，并将其放置在漏斗图内部。
     - `itemstyle_opts=opts.ItemStyleOpts(border_color="#fff", border_width=1)`: 设置漏斗图项的样式，包括边框颜色和宽度。
   - 使用 `.set_global_opts()` 设置全局选项，包括标题。
   - 使用 `.render_notebook()` 方法在 Jupyter Notebook 中渲染图表。

## 3.3 用户特征分析
### 购买率高用户特征

购买率高人群特征(按购买率倒序排列)

In [18]:
df_pivot['购买率']=df_pivot['buy']/df_pivot[df_pivot['pv']>0]['pv']  #没设置成%
df_pivot.sort_values(by='购买率',ascending=False)  #购买率高人群特征(按购买率倒序排列)

Behavior_type,buy,cart,fav,pv,购买率
User_Id,,,,,
1005709,15,4,0,1,15.0
129771,10,5,2,1,10.0
202754,9,4,47,1,9.0
18858,8,15,7,1,8.0
100678,11,11,0,2,5.5
...,...,...,...,...,...
1004458,12,6,1,0,NaN
1005688,7,46,0,0,NaN
1005897,3,16,13,0,NaN


- `df_pivot[df_pivot['pv'] > 0]['pv']`：这部分代码首先筛选出所有 `pv`（页面浏览量）大于 0 的记录，然后获取这些记录中的 `pv` 列值。这里需要注意的是，由于除法运算，为了避免除以 0 的情况，只选择 `'pv'` 值大于 0 的行来进行计算。

购买率高人群特征(按购买数倒序排列)

In [17]:
df_pivot.sort_values(by='buy',ascending=False)

Behavior_type,buy,cart,fav,pv
User_Id,,,,
107932,72,16,0,139
122504,69,0,0,48
128379,65,0,1,0
190873,61,0,0,0
1008380,57,28,0,127
...,...,...,...,...
1017970,0,1,0,7
154423,0,3,0,266
1017849,0,1,0,25


由以上结果可以看出，购买率有很多大于100%的情况，这部分猜测是因为数据取值区间的问题。一些用户之前已经把商品加入购物车，然后直接付款买了，所以没产生点击。
不过实际情况如果属于异常数据，在数据清洗的时候，也可以把这些数据剔除再进行分析。

不过从上面数据也大致能看到，购买率高的用户点击率反而不怎么高，这些用户收藏数和加购物车的次数也很少，这部分用户应该属于理智型消费者，有明确的购物目标，属于缺啥买啥型，不容易被店家广告或促销吸引。


### 购买率低的用户特征

In [19]:
df_pivot.sort_values(by='buy')

Behavior_type,buy,cart,fav,pv,购买率
User_Id,,,,,
1017928,0,9,1,41,0.000000
1017931,0,0,2,130,0.000000
134805,0,0,0,9,0.000000
134635,0,10,1,229,0.000000
134670,0,1,0,11,0.000000
...,...,...,...,...,...
1008380,57,28,0,127,0.448819
190873,61,0,0,0,NaN
128379,65,0,1,0,NaN


由以上结果可以看出，购买数比较少的用户分为两类，一类是点击次数少的，一方面的原因是这类用户可能不太会购物或者不喜欢上网的用户，可以加以引导，另一方面是从商品的角度考虑，是否商品定价过高或设计不合理；第二类用户是点击率高、收藏或加购物车也多的用户，此类用户可能正为商家的促销活动做准备，下单欲望较少且自制力较强，思虑多或者不会支付，购物难度较大。


## 3.4 基于时间维度了解用户的行为习惯
### 一天中用户的活跃时段分布

In [20]:
pd.to_datetime(df['Date']).dt.day_name()  #转换成日期格式后,用.dt的day_name()函数转为英文星期

0           Saturday
1           Saturday
2           Saturday
3           Saturday
4           Saturday
             ...    
2998527    Wednesday
2998528    Wednesday
2998529    Wednesday
2998530    Wednesday
2998531    Wednesday
Name: Date, Length: 2998532, dtype: object

这段代码是用来将 DataFrame (`df`) 中的 `'Date'` 列转换为日期时间对象，并从中提取每一天的英文名称。

1. **`pd.to_datetime(df['Date'])`**:
   - `pd.to_datetime(...)` 是 Pandas 中的一个函数，用于将各种日期时间相关的数据转换为 datetime 类型的对象。
   - `df['Date']` 是 DataFrame `df` 中名为 `'Date'` 的列。
   - 这行代码将 `'Date'` 列中的数据转换为 datetime 对象。

2. **`.dt.day_name()`**:
   - `.dt` 是 Pandas 中用于访问 datetime 属性的方法。
   - `.day_name()` 是一个方法，用于获取每个 datetime 对象所代表的星期几的英文名称。

In [21]:
df['星期几'] = pd.to_datetime(df['Date']).dt.day_name()
df

,User_Id,Item_Id,Category_Id,Behavior_type,Date,Time,星期几
0,1,2268318,2520377,pv,2017-11-25,01:21:10,Saturday
1,1,2333346,2520771,pv,2017-11-25,06:15:33,Saturday
2,1,2576651,149192,pv,2017-11-25,09:21:25,Saturday
3,1,3830808,4181361,pv,2017-11-25,15:04:53,Saturday
4,1,4365585,2520377,pv,2017-11-25,15:49:06,Saturday
...,...,...,...,...,...,...,...
2998527,218275,2772813,1577687,pv,2017-11-29,23:10:10,Wednesday
2998528,218275,4654492,1577687,pv,2017-11-29,23:11:08,Wednesday
2998529,218275,2772813,1577687,pv,2017-11-29,23:11:17,Wednesday
2998530,218275,1207527,1577687,pv,2017-11-29,23:12:50,Wednesday


In [23]:
df['hours'] = df['Time'].str.split(':',n=1,expand=True)[0]
df

,User_Id,Item_Id,Category_Id,Behavior_type,Date,Time,星期几,hours
0,1,2268318,2520377,pv,2017-11-25,01:21:10,Saturday,01
1,1,2333346,2520771,pv,2017-11-25,06:15:33,Saturday,06
2,1,2576651,149192,pv,2017-11-25,09:21:25,Saturday,09
3,1,3830808,4181361,pv,2017-11-25,15:04:53,Saturday,15
4,1,4365585,2520377,pv,2017-11-25,15:49:06,Saturday,15
...,...,...,...,...,...,...,...,...
2998527,218275,2772813,1577687,pv,2017-11-29,23:10:10,Wednesday,23
2998528,218275,4654492,1577687,pv,2017-11-29,23:11:08,Wednesday,23
2998529,218275,2772813,1577687,pv,2017-11-29,23:11:17,Wednesday,23
2998530,218275,1207527,1577687,pv,2017-11-29,23:12:50,Wednesday,23


这段代码是用来从 DataFrame (`df`) 中的 `'Time'` 列中提取小时部分，并将其存储在一个新的列 `'hours'` 中。

**`df['Time'].str.split(':', 'n=1', expand=True)`**:
   - `.str` 是 Pandas Series 中用于字符串操作的方法。
   - `.split(':', 'n=1', expand=True)` 是一个方法，用于将字符串按照指定的分隔符进行分割。
     - `':'` 是分隔符，这里用来分割时间字符串。
     - `n=1` 表示最多进行一次分割，这样只会分割第一个出现的 `':'`。
     - `expand=True` 表示将分割的结果扩展为一个 DataFrame，其中每一列对应分割后的一个部分。

In [27]:
hours_pivot = df.pivot_table(values='Item_Id',index='hours',columns='Behavior_type',aggfunc='count',margins=True)
hours_pivot

Behavior_type,buy,cart,fav,pv,All
hours,,,,,
00,1761,5601,3099,91408,101869
01,699,2599,1337,41624,46259
02,388,1422,890,23346,26046
03,179,1061,518,15987,17745
04,225,839,484,14018,15566
05,223,1156,603,16605,18587
06,531,2290,1318,34605,38744
07,1117,4419,2261,67209,75006
08,1913,5805,3156,92302,103176


In [29]:
#1.一天中用户的活跃时段分布
hours_pivot = df.pivot_table(values='Item_Id',index='hours',columns='Behavior_type',aggfunc='count',margins=True)
from pyecharts.charts import Bar
from pyecharts import options as opts

x =  hours_pivot.index[:-1].tolist()
y1 = hours_pivot.buy[:-1].tolist()
y2 = hours_pivot.cart[:-1].tolist()
y3 = hours_pivot.fav[:-1].tolist()
y4 = hours_pivot.pv[:-1].tolist()
bar = (Bar()
      .add_xaxis(x)
      .add_yaxis('购买数',y1)
      .add_yaxis('加购数',y2)
      .add_yaxis('收藏次数',y3)
      .add_yaxis('点击次数',y4)
      .set_global_opts(title_opts=opts.TitleOpts(title='一天中用户活跃情况'),
                       xaxis_opts=opts.AxisOpts(name='小时'))
      .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
      )


bar.render_notebook()

1. **创建透视表**:
   ```python
   hours_pivot = df.pivot_table(values='Item_Id', index='hours', columns='Behavior_type', aggfunc='count', margins=True)
   ```
   - 创建一个透视表 `hours_pivot`，其中：
     - `margins=True` 添加总计行和总计列，方便查看所有行为类型的总和以及所有小时的总和。

2. **导入相关库**:
   ```python
   from pyecharts.charts import Bar
   from pyecharts import options as opts
   ```
   - 从 `pyecharts.charts` 模块导入`Bar` 类，`Bar` 类既可以生成柱状图（垂直条形图），也可以生成条形图（水平条形图）。

2. **准备绘图数据**:
   ```python
   x = hours_pivot.index[:-1].tolist()
   y1 = hours_pivot.buy[:-1].tolist()
   y2 = hours_pivot.cart[:-1].tolist()
   y3 = hours_pivot.fav[:-1].tolist()
   y4 = hours_pivot.pv[:-1].tolist()
   ```
   - `hours_pivot.index[:-1].tolist()` 获取透视表的行索引（小时）并转换为列表，不包括最后一个汇总行。
   - `hours_pivot.buy[:-1].tolist()`, `hours_pivot.cart[:-1].tolist()`, `hours_pivot.fav[:-1].tolist()`, `hours_pivot.pv[:-1].tolist()` 分别获取 `'buy'`, `'cart'`, `'fav'`, `'pv'` 列的数据并转换为列表，同样不包括最后一个汇总行。

3. **创建条形图**:
   ```python
   bar = (Bar()
         .add_xaxis(x)
         .add_yaxis('购买数', y1)
         .add_yaxis('加购数', y2)
         .add_yaxis('收藏次数', y3)
         .add_yaxis('点击次数', y4)
         .set_global_opts(
             title_opts=opts.TitleOpts(title='一天中用户活跃情况'),
             xaxis_opts=opts.AxisOpts(name='小时')
         )
         .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
         )
   ```
   - 使用 `Bar` 类创建一个条形图实例。
   - 使用 `.add_xaxis(x)` 添加 X 轴数据。
   - 使用 `.add_yaxis(...)` 多次调用以添加多个 Y 轴数据系列，分别为 `'购买数'`, `'加购数'`, `'收藏次数'`, 和 `'点击次数'`。
   - 使用 `.set_global_opts(...)` 设置全局选项，包括标题和 X 轴名称。
   - 使用 `.set_series_opts(...)` 设置系列选项，这里指定了不显示标签。

4. **渲染条形图**:
   ```python
   bar.render_notebook()
   ```
   - 使用 `.render_notebook()` 方法在 Jupyter Notebook 中渲染图表。

###  一周中用户活跃时段分布
数据截取了11-27到12-03一个完整周内的数据

In [30]:
df3 = df[(df['Date']>='2017-11-27')&(df['Date']<='2017-12-03')]
df3

,User_Id,Item_Id,Category_Id,Behavior_type,Date,Time,星期几,hours
7,1,3827899,2920476,pv,2017-11-27,00:24:33,Monday,00
8,1,3745169,2891509,pv,2017-11-27,03:44:31,Monday,03
9,1,1531036,2920476,pv,2017-11-27,06:02:12,Monday,06
10,1,2266567,4145813,pv,2017-11-27,08:11:11,Monday,08
11,1,2951368,1080785,pv,2017-11-27,10:47:08,Monday,10
...,...,...,...,...,...,...,...,...
2998527,218275,2772813,1577687,pv,2017-11-29,23:10:10,Wednesday,23
2998528,218275,4654492,1577687,pv,2017-11-29,23:11:08,Wednesday,23
2998529,218275,2772813,1577687,pv,2017-11-29,23:11:17,Wednesday,23
2998530,218275,1207527,1577687,pv,2017-11-29,23:12:50,Wednesday,23


In [32]:
week_pivot = df3.pivot_table(values='User_Id',index=df3['星期几'],columns='Behavior_type',aggfunc='count')
week_pivot

Behavior_type,buy,cart,fav,pv
星期几,,,,
Friday,6322,19094,9188,294298
Monday,6717,16666,8605,267000
Saturday,7543,23872,12038,370282
Sunday,7773,23842,12044,371188
Thursday,6718,16909,8891,280678
Tuesday,6357,16275,8602,265157
Wednesday,6661,16487,8816,271342


In [35]:
week_pivot['day of week'] = np.array([5,1,6,7,4,2,3]).reshape(7,1)
week_pivot_new = week_pivot.sort_values(['day of week'],ascending=True)
week_pivot_new

Behavior_type,buy,cart,fav,pv,day of week
星期几,,,,,
Monday,6717,16666,8605,267000,1
Tuesday,6357,16275,8602,265157,2
Wednesday,6661,16487,8816,271342,3
Thursday,6718,16909,8891,280678,4
Friday,6322,19094,9188,294298,5
Saturday,7543,23872,12038,370282,6
Sunday,7773,23842,12044,371188,7


这段代码是用来对透视表 `week_pivot` 进行排序，使得星期几按照从星期一到星期日的顺序排列。

- **`np.array([5, 1, 6, 7, 4, 2, 3])`**:
  - 使用 NumPy 的 `array` 函数创建一个一维数组，包含星期几的顺序，这里的顺序是从星期日开始到星期六结束。注意这里的顺序与通常的星期一到星期日的顺序不同，因此需要根据实际情况调整。
    
- **`.reshape(7, 1)`**:
  - 将一维数组转换为二维数组，形状为 7 行 1 列。这是因为 `week_pivot` 的行索引也是 7 个星期几的名称。

一周中用户活跃情况

In [36]:
from pyecharts.charts import Bar
from pyecharts import options as opts

x =  week_pivot_new.index.tolist()
y1 = week_pivot_new.buy.tolist()
y2 = week_pivot_new.cart.tolist()
y3 = week_pivot_new.fav.tolist()
y4 = week_pivot_new.pv.tolist()
bar = (Bar()
      .add_xaxis(x)
      .add_yaxis('购买数',y1)
      .add_yaxis('加购数',y2)
      .add_yaxis('收藏次数',y3)
      .add_yaxis('点击次数',y4)
      .set_global_opts(title_opts=opts.TitleOpts(title='一周中用户活跃情况'),
                       xaxis_opts=opts.AxisOpts(name='周'))
      .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
      )


bar.render_notebook()

由以上结果可以看出，每周用户活跃度较稳定，周末会有大幅度的提高。但从购买数来看，周五会有小幅下滑。

# 总结
1、总体转化率只有2.24%，用户点击后收藏加购物车的转化率为9.46%，需要提高用户的购买意愿，可通过活动促销、精准营销等方式。

2、购买率高且点击量少的用户属于理智型购物者，有明确购物目标，受促销和广告影响少；而购买率低的用户可以认为是等待型或克制性群体，下单欲望较少且自制力较强，购物难度较大。

3、大部分用户的主要活跃时间在10点到23点，在20点到22点达到一天的顶峰。每周用户活跃度比较平稳，周末显著升高。可以根据用户的活跃时间精准推送商家的折扣优惠或促销活动，提高购买率。
